In [1]:
%run "0_workspace_setup.ipynb"

In [2]:
from nhm_helpers.output_plots import create_streamflow_plot
from nhm_helpers.nhm_hydrofabric import make_hf_map_elements
from nhm_helpers.map_template import make_streamflow_map
from nhm_helpers.nhm_output_visualization import retrieve_hru_output_info
from ipywidgets import widgets
from IPython.display import display

poi_id_sel = None
crs = 4326

## Introduction
The purpose of this notebook is to assist in the evaluation of NHM subdomain model simulated streamflow. First, the notebook creates a map of gage locations color coded by Kling-Gupta efficiency (KGE; [2009](https://www.sciencedirect.com/science/article/abs/pii/S0022169409004843)) value. Gage locations are overlays in a map of NHM headwater basins (HWs) that are color coded to calibration type: yellow indicates HWs calibrated with statistical streamflow targets at the HW outlet; green indicates HWs that were further calibrated with streamflow observations at selected gage locations.
Next, a gage may be selected and a plot created that shows a time-series of simulated and observed streamflow at daily, monthly, and annual time steps. A flow exceedance curve and table of summary statistics is also provided in the plot for streamflow evaluation purposes. Additionally, maps and plots produced are saved for use outside of notebooks as HTML files in the `html_maps` and `html_plots` folders in the `notebook_output` folder.

The cell below reads the NHM subdomain model hydrofabric elements for mapping purposes using `make_hf_map_elements()`, retrieves `pywatershed` output file information for mapping and plotting using `retrieve_hru_output_info()`, and writes general NHM subdomain model run and hydrofabric information.

In [3]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)
con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

NHM model domain: walla_walla, parameter file: myparam.param
Simulation and observation data range: 01/01/2011 - 12/31/2022 (from control.default.bandit).
 
The parameter file contains 8 gages managed by USGS, 44 segments, and 87 HRUs. 
     85 HRUs are within HWs, and 55 are within HW calibrated with streamflow observations.

## Create an interactive map to evaluate streamflow at gages (pois)
The following cell creates a map of the NHM subdomain model hydrofabric elements and displays monthly Kling-Gupta efficiency (KGE) values for parameter file gages as red (KGE<0.5), yellow (0.5<=KGE<0.7), and green (KGE>=0.7). The map is interactive, made with [folium](https://python-visualization.github.io/folium/v0.18.0/index.html) (see [README](./README.md) for basic map interactive functionality). Use the mouse to left-click on gage icons and display the gage id and gage name. Use the mouse to left-click on HRUs to display the headwater basin (HW) id. HW groupings of HRUs are important when interpreting parameter values and output variables. HRU parameters were grouped by HW and adjusted using a scaling factor during the byHW and byHWobs parts of NHM calibration ([Hay and others, 2023](https://pubs.usgs.gov/tm/06/b10/tm6b10.pdf)). During the byHW calibration part, statistical flow targets at HW outlets were used, and during the subsequent byHWobs part, streamflow obervations were used for selected HWs.

In [4]:
map_file = make_streamflow_map(
    out_dir,
    plot_start_date,
    plot_end_date,
    water_years,
    hru_gdf,
    poi_df,
    poi_id_sel,
    seg_gdf,
    html_maps_dir,
    subdomain,
    HW_basins_gdf,
    HW_basins,
    output_netcdf_filename,
)

KeyError: 'npoi_gages'

In [8]:
poi_df

,poi_id,poi_gage_segment,poi_type,nhm_seg,poi_agency,poi_name,latitude,longitude,drainage_area,drainage_area_contrib,nhm_calib
0,14018500,1,1,49698,USGS,"WALLA WALLA RIVER NEAR TOUCHET, WA",46.027609,-118.729689,1657.0,NaN,N
1,14010000,11,1,49708,USGS,"SOUTH FORK WALLA WALLA RIVER NEAR MILTON,OREG.",45.829857,-118.169950,63.0,NaN,N
2,14010800,13,0,49710,USGS,NORTH FRK WALLA WALLA RIVER NR MILTON FREEWATE...,45.884580,-118.202731,34.4,NaN,N
3,14015000,16,1,49713,USGS,"MILL CREEK AT WALLA WALLA, WA",46.076248,-118.273572,95.7,NaN,Y
4,14013700,17,1,49714,USGS,"MILL CREEK AT FIVE MILE RD BR NR WALLA WALLA, WA",46.085693,-118.228291,92.7,NaN,N
5,14013000,19,1,49716,USGS,"MILL CREEK NEAR WALLA WALLA, WA",46.007893,-118.118559,59.6,NaN,Y
6,14017000,29,1,49726,USGS,"TOUCHET RIVER AT BOLLES, WA",46.274307,-118.221904,361.0,NaN,N
7,14013500,39,1,49736,USGS,"BLUE CREEK NEAR WALLA WALLA, WA",46.057638,-118.140228,17.0,NaN,N


In [11]:
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display, clear_output, IFrame

# ─── 1. Define widgets ───────────────────────────────────────────────────
gage_txt = widgets.Text(
    description="Streamgage ID:",
    placeholder="(optional) Enter gage id",
    layout=widgets.Layout(width="40%"),
    style={"description_width": "initial"},
)

btn_plot = widgets.Button(
    description="Generate Plot",
    button_style="primary",
)

btn_map = widgets.Button(
    description="Generate Map",
    button_style="primary",
)

controls = widgets.HBox([gage_txt, btn_plot, btn_map])

# ─── 2. Output areas ─────────────────────────────────────────────────────
plot_out = widgets.Output()
map_out = widgets.Output()


# ─── 3. Callback functions ───────────────────────────────────────────────
def on_plot_clicked(b: widgets.Button) -> None:
    """
    When clicked, clear previous plot, default to first POI if none entered,
    then generate and display the streamflow plot.
    """
    with plot_out:
        clear_output(wait=True)
        poi_id_sel = gage_txt.value.strip() or poi_df.poi_id.tolist()[0]
        fplot = create_streamflow_plot(
            poi_id_sel,
            plot_start_date,
            plot_end_date,
            water_years,
            html_plots_dir,
            output_netcdf_filename,
            out_dir,
            subdomain,
        )
        display(fplot)


def on_map_clicked(b: widgets.Button) -> None:
    """
    When clicked, clear previous map, default to first POI if none entered,
    then generate and display the streamflow map.
    """
    with map_out:
        clear_output(wait=True)
        poi_id_sel = gage_txt.value.strip() or poi_df.poi_id.tolist()[0]
        map_file = make_streamflow_map(
            out_dir,
            plot_start_date,
            plot_end_date,
            water_years,
            hru_gdf,
            poi_df,
            poi_id_sel,
            seg_gdf,
            html_maps_dir,
            subdomain,
            HW_basins_gdf,
            HW_basins,
            output_netcdf_filename,
        )
        if isinstance(map_file, str):
            display(IFrame(src=map_file, width="100%", height="500px"))
        else:
            display(map_file)


# ─── 4. Wire up buttons ─────────────────────────────────────────────────
btn_plot.on_click(on_plot_clicked)
btn_map.on_click(on_map_clicked)

# ─── 5. Display everything ───────────────────────────────────────────────
display(widgets.VBox([controls, plot_out, map_out]))

## Make an interactive plot of simulated and observed streamflow and table of streamflow statistics for selected gage
This plot is interactive, and made with [plotly](https://plotly.com/) (see [README](./README.md) for basic plot interactive functionality), and shows subplots of simulated and observed streamflow at daily, monthly, and annual timesteps for the selected gage. Note: the date that appears in popup window when hovering over plotted data reflects the last day of the timestep displayed in the plot. A flow exceedance curve and table of summary statistics is also provided in the plot for streamflow evaluation purposes. 

## Select a gage to evaluate simulated and observed streamflow time-series and streamflow statistics
<font size=4> &#x270D;<font color='green'>**Enter Information:**</font> **Run the cell below. In the resulting drop-down box, select a gage. A plot will be created to evaluate observed and simulated streamflow at the selected gage.** </font><br>
<font size=3> If no gage is selected (default), the first gage listed in the parameter file will be used. The drop-down box selection can be changed and additional plots will be displayed and exported (html files) to<br> `<NHM subdomain model folder>/notebook_output_files/html_plots`.

In [10]:
display(widgets.VBox([controls, plot_out, map_out]))